In [9]:
import glob
import os
import pickle as pkl
import re
import sys

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
with open("nrel_clean.pkl", "rb") as f:
    nrel_db = pkl.load(f)

In [4]:
features = list(filter(lambda x: len(x) > 0, nrel_db.keys()))
relevant_features = features[1:-6]
print(f"N Features: {len(features)}")
print(f"N Relevant Features: {len(relevant_features)}")
relevant_features

N Features: 30
N Relevant Features: 23


['sorted formula     sorted chemical formula',
 'ΔHf (eV/atom)     final enthalpy, eV/atom',
 'Stb     qhull stability',
 'ΔHGS enthalpy wrt ground state',
 'ΔHDecomp enthalpy wrt decomp',
 'ΔEGS energyperatom wrt ground state',
 'ICSD SG     ICSD space group number',
 'final SG     final space group number',
 'Etot (eV/Atom)     total energy, per atom, eV/Atom',
 'minID     id having min energy for this formula',
 'ΔEcbm     gwvd: Ecbm - parentEcbm: difference in conduction band minimum energy, eV',
 'ΔEvbm     gwvd: Evbm - parentEvbm: difference in valence band maximum energy, eV',
 'εe(IP)     static electronic dielectric constant (independent particle)',
 'εe(TDDFT)     static electronic dielectric constant (TD-DFT)',
 'εe(post-lopt)     static electronic dielectric constant (from post-lopt)',
 'Eg (eV)     bandgap, eV',
 'Eg,d (eV)     direct bandgap, eV',
 'me*/m0 DOS effective mass for electrons, T=1000K',
 'mh*/m0 DOS effective mass for holes, T=1000K',
 'netCharge     net char

In [7]:
nonzero_idxs = dict()
for k in nrel_db.columns:
    x = nrel_db[k]
    nonzero_idxs[k] = x[x.notna()].index

In [8]:
nz_lens = dict()
for k in nonzero_idxs.keys():
    nz_lens[k] = len(nonzero_idxs[k])
pd.Series(nz_lens)

id     MatDB ID                                                                        1639
sorted formula     sorted chemical formula                                             1639
ΔHf (eV/atom)     final enthalpy, eV/atom                                               991
Stb     qhull stability                                                                 991
ΔHGS enthalpy wrt ground state                                                          991
ΔHDecomp enthalpy wrt decomp                                                            991
ΔEGS energyperatom wrt ground state                                                     991
ICSD SG     ICSD space group number                                                    1318
final SG     final space group number                                                  1639
Etot (eV/Atom)     total energy, per atom, eV/Atom                                     1592
minID     id having min energy for this formula                                 

In [33]:
nrel_preusable = nrel_db.loc[nrel_db['εe(post-lopt)     static electronic dielectric constant (from post-lopt)'].notna()]
useable_columns = (nrel_preusable.notna()).all(axis=0)

In [36]:
nrel_usable = nrel_preusable.loc[:, useable_columns.values]

15

In [ ]:
eps_IP = nrel_useable['εe(IP)     static electronic dielectric constant (independent particle)']


In [11]:
from sklearn import linear_model

In [ ]:
reg = linear_model.LassoCV()
reg.fit()